## Finding entity classes in embeddings

In this notebook we're going to use embeddings to find entity classes and how they correlate with other things

In [ ]:
%matplotlib inline
from sklearn import svm
from keras.utils import get_file
import os
import gensim
import numpy as np
import random
import requests
import geopandas as gpd
from IPython.core.pylabtools import figsize
figsize(12, 8)
import csv

In [ ]:
MODEL    = 'GoogleNews-vectors-negative300.bin'
data_loc = '/home/smithw/Downloads/deep_learning' # WS: files not backed up here
zipped   = os.path.join(data_loc, MODEL + '.gz')  # WS mod
unzipped = os.path.join(data_loc, MODEL)  # WS
zipped, unzipped

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format(unzipped, binary=True)

In [ ]:
type(model), type(model.vectors)

In [ ]:
model.most_similar(positive=['Germany'])

In [ ]:
model.most_similar(positive=['Annita_Kirsten'])

In [ ]:
countries = list(csv.DictReader(open('data/countries.csv')))
len(countries), countries[:10]

In [ ]:
len(model.key_to_index), len(model.index_to_key)

## TRAIN SVM ON COUNTRY NAMES vs NON-COUNTRY NAMES

In [ ]:
len(countries), len(model)

In [ ]:
# examples of country names for training
num_countries = 140
positive = [x['name'] for x in random.sample(countries, num_countries)]
positive[:10]

In [ ]:
# exampls of not-country names
sample = 10000 # WS note: this may randomly contain some countries, but unlikely out of 3e6 tokens
negative = random.sample(model.index_to_key, sample)  # WS this works
negative[:10]

In [ ]:
# now label the data
labelled = [(p, 1) for p in positive] + [(n, 0) for n in negative]
random.shuffle(labelled)
X = np.asarray([model[w] for w, l in labelled])
y = np.asarray([l        for w, l in labelled])
X.shape, y.shape

In [ ]:
labelled[:5]

In [ ]:
sum(y) == num_countries  # add up all the truth: should be same as num_countries

In [ ]:
TRAINING_FRACTION = 0.1
cut_off = int(TRAINING_FRACTION * len(labelled))
clf     = svm.SVC(kernel='linear')  # support-vector classification
clf.fit(X[:cut_off], y[:cut_off])  # training

In [ ]:
#svm.SVC?, svm.SVC.fit?
#svm.SVC.predict?

In [ ]:
# get the predictions from the validation set
res = clf.predict(X[cut_off:])

In [ ]:
# test on validation data
missed = [(country, pred) for (pred, truth, country) in zip(res, y[cut_off:], labelled[cut_off:]) if pred != truth]  # WS mod
100 - 100 * float(len(missed)) / len(res), missed

In [ ]:
def evaluate(truth, pred):  # WS creation
    total = truth.shape[0]
    tab = {(1, 1): 'TRUE  POSITIVE', (1, 0): 'FALSE  NEGATIVE', 
           (0, 1): 'FALSE POSITIVE', (0, 0): 'TRUE   NEGATIVE'}
    score = {(1, 1): 0, (0, 0): 0, (1, 0): 0, (0, 1): 0}
    for t, p in zip(truth, pred): score[(t, p)] += 1
    truth_table(tab, score)
    correct = score[(0, 0)] + score[(1, 1)]
    print(f'\nTRUE NEG plus TRUE POS: {correct:5} ({float(100 * correct / total):7.4f}%)')

In [ ]:
def truth_table(name, values, width=55):  # WS
    print(f'\n{name[(1, 1)]:14}: {values[(1, 1)]:5}   |   {name[(1, 0)]:14}: {values[(1, 0)]:5}')
    print(f'{"".join("_"*width)}')
    print(f'\n{name[(0, 1)]:14}: {values[(0, 1)]:5}   |   {name[(0, 0)]:14}: {values[(0, 0)]:5}\n')

In [ ]:
evaluate(y[:cut_off], clf.predict(X[:cut_off]))  # training sample

In [ ]:
evaluate(y[cut_off:], clf.predict(X[cut_off:]))  # validation sample

## LOOK AT SAMPLES OF ALL THINGS CALLED A COUNTRY IN A LARGER SET

In [ ]:
model.vectors.shape

In [ ]:
# note: if too many vectors are taken, RAM will overflow, given that the word2vec dbase is also in RAM
# the full 3000000 is too large to take at once
#all_predictions = clf.predict(model.vectors[:1000000]) # 1e6 takes 30s to run
batch_size = 100000  # takes 3 seconds for 100000
batch_predictions = clf.predict(model.vectors[:batch_size, :]) # WS mod

In [ ]:
batch_predictions.shape, sum(batch_predictions)

In [ ]:
type(model.index_to_key)

In [ ]:
model.index_to_key[:10]

In [ ]:
res = []
# zip will go to the smallest of the length of batch_predictions or model.index_to_key
for word, pred in zip(model.index_to_key, batch_predictions):  # WS index_to_key replaces index2word
    if pred:
        res.append(word)  # WS turned off break: see how many hits there are
        #if len(res) == 150:
        #    break
random.sample(res, 20) # can see the country false-alarms mixed in with the true countries

In [ ]:
len(res)

# START HERE

In [ ]:
country_to_idx = {country['name']: idx for idx, country in enumerate(countries)}

In [ ]:
countries[0]

In [ ]:
country_names = [k['name'] for k in countries]
country_names.sort()
#country_names

In [ ]:
country_vecs = np.asarray([model[c['name']] for c in countries])  # get vector for each country
country_vecs.shape

Quick sanity check to see what is similar to Canada:

In [ ]:
dists = np.dot(country_vecs, country_vecs[country_to_idx['Canada']])  # (184, 300) dot (300,) => (184,)

In [ ]:
country_vecs[0].shape, dists.shape

In [ ]:
for idx in reversed(np.argsort(dists)[-10:]):
    print(countries[idx]['name'], dists[idx])

Ranking countries for a specific term:

In [ ]:
def rank_countries(term, topn=10, field='name'):
    if not term in model:
        return []
    vec = model[term]
    dists = np.dot(country_vecs, vec)
    return [(countries[idx][field], float(dists[idx])) for idx in reversed(np.argsort(dists)[-topn:])]

In [ ]:
rank_countries('cricket', field='name')  # field also can be 'cc' or 'cc3'

Now let's visualize this on a world map:

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head(20)

We can now plot some maps!

In [ ]:
type(world)

In [ ]:
#world['iso_a3'].map?

In [ ]:
def map_term(term):
    d = {k.upper(): v for k, v in rank_countries(term, topn=0, field='cc3')}
    print(len(d))
    if len(d) > 0:  # WS added to handle empty results
        world[term] = world['iso_a3'].map(d)  # WS this creates a new column for the dataframe 'world'
        world[term] /= world[term].max()
        world.dropna().plot(term, cmap='OrRd')

In [ ]:
map_term('United_States')

In [ ]:
print(world.columns)

In [ ]:
world.head(5)

In [ ]:
world.dropna().plot('name', ) #cmap='OrRd')

In [ ]:
world.dropna().plot('continent')

In [ ]:
world.dropna().plot('gdp_md_est', cmap='prism')

In [ ]:
#world.plot?

In [ ]:
map_term('coffee')

In [ ]:
print(world['coffee'].dropna())

In [ ]:
map_term('cricket')

In [ ]:
map_term('China')

In [ ]:
map_term('vodka')

In [ ]:
map_term('guns')

In [ ]:
map_term('Panama_Canal')

In [ ]:
map_term('Paris')

In [ ]:
map_term('cancer')

In [ ]:
map_term('accidents')

In [ ]:
map_term('murder')

In [ ]:
map_term('poor')

In [ ]:
map_term('G7')

In [ ]:
map_term('wealthy')

In [ ]:
map_term('paradise')

In [ ]:
map_term('WWII')